In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os
from IPython.display import display, clear_output

import glob

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.linear_model import LassoCV

In [5]:
pfam_table_columns = ['target_name','target_accession','query_name','query_accession', 
                      'full_e_value', 'full_score', 'full_bias', 
                      'best_domain_e_value', 'best_domain_score', 'best_domain_bias', 
                      'exp', 'reg', 'clu',  'ov', 'env', 'dom', 'rep', 'inc', 'description_of_target']


def read_pfam_table_pandas(file_path : str) -> pd.DataFrame:
    """
    file_path: path to pfam "tblout"
    """
    # read table with special parameters
    pfam_as_table = pd.read_table(file_path, skiprows=3,  skipfooter=10, sep='\s+', header=None, engine='python')
    # un-split last 6 columns
    description = pfam_as_table.iloc[:, 18:].apply(lambda row: ''.join([str(v).lstrip() for v in row.values]), axis=1)
    # remove last 6 columns, add new one of joined values
    pfam_as_table = pfam_as_table.iloc[:, :18].assign(description_of_target=description.values)
    # reset column names
    pfam_as_table.columns = pfam_table_columns
    
    return pfam_as_table

In [10]:
globbed_pfam_search_results = glob.glob('/data/mhoffert/genomes/GTDB_r207/pfam/*tbl*')
pfam_data = pd.read_csv('/data/mhoffert/db/pfam/pfamA.txt.gz', sep='\t', header=None, index_col=0)

In [ ]:
    pfam_table_scores = pd.merge(pfam_table, pfam_data.loc[interesting_pfams, 9], how='left', left_on='Pfam', right_index=True)
    # get scores better than trusted thresholds
    final_table = pfam_table_scores[pfam_table_scores['full_score'] >= pfam_table_scores[9]]

In [24]:
all_pfam_data = []
num_done = 0
for curr_genome_pfam in globbed_pfam_search_results:
    
    g = curr_genome_pfam.split('/')[-1].split('_tbl')[0]
    
    display(num_done)
    clear_output(wait=True)
    # curr_genome_pfam = f'/data/mhoffert/genomes/GTDB_r207/pfam/{g}_tblout.txt'
    
    # read the HMMER results as a table
    pfam_as_table = read_pfam_table_pandas(curr_genome_pfam)
    
    pfam_as_table['Pfam'] = pfam_as_table['query_accession'].apply(lambda x: x.split('.')[0])
    
    pfam_table_scores = pd.merge(pfam_as_table, pfam_data.loc[:, 9], how='left', left_on='Pfam', right_index=True)
    # get scores better than trusted thresholds
    final_table = pfam_table_scores[pfam_table_scores['full_score'] >= pfam_table_scores[9]]
    
    # get the pfams found
    pfams = final_table['Pfam'].unique()
    all_pfam_data.append(pd.Series(index=pfams, data=[1]*len(pfams), dtype=int, name=g))
    num_done += 1

62290

In [26]:
all_pfam_df = pd.concat(all_pfam_data, axis=1).fillna(0).T

In [27]:
all_pfam_df.shape

(62291, 17422)

In [28]:
all_pfam_df.to_csv('/data/mhoffert/genomes/GTDB_r207/gtdb_genomes_reps_r207_pfam_trusted.tsv.gz', sep='\t', compression='gzip')